## Understanding Welfare, Gentrification and Wealth in California

Author: Aakash Pattabi

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

AttributeError: module 'matplotlib' has no attribute 'cbook'

### Load data and identify key variables

We'll begin by manipulating only one of the data files, namely the most recent one including American FactFinder data for Census Tracts in California in the year 2016. This is based on one-year Census estimates conducted between the decennial Censuses. Notably, the measurements are estimated, and margins of error are provided. 

In [ ]:
df_2016_path = "B25056_contract_rent_2013-2015/ACS_16_5YR_S2503/ACS_16_5YR_S2503_with_ann.csv"
df_2016 = pd.read_csv(df_2016_path, header = 0, skiprows = [1])

print(df_2016["GEO.id"].head(5))

We'll focus specifically on how renters are doing first, because owners in California are given special tax protections under Proposition 13, which means that even owners with low incomes may still be able to afford their homes (due to fixed low property taxes). They key columns are:

- HC03_EST_VC01: Estimate of renter-occupied housing units
- HC03_EST_VC03: Estimate of renter-occupied units with income < \$5,000 in last year
- HC03_EST_VC04: Estimate of renter-occupied units with income \$5,000-9,999 in last year
- HC03_EST_VC05: Estimate of renter-occupied units with income \$10,000-14,999 in last year
- HC03_EST_VC06: Estimate of renter-occupied units with income \$15,000-19,999 in last year
- HC03_EST_VC07: Estimate of renter-occupied units with income \$20,000-24,999 in last year
- HC03_EST_VC08: Estimate of renter-occupied units with income \$25,000-34,999 in last year
- HC03_EST_VC09: Estimate of renter-occupied units with income \$35,000-49,999 in last year
- HC03_EST_VC10: Estimate of renter-occupied units with income \$50,000-74,999 in last year
- HC03_EST_VC11: Estimate of renter-occupied units with income \$75,000-99,999 in last year
- HC03_EST_VC12: Estimate of renter-occupied units with income \$100,000-149,999 in last year
- HC03_EST_VC13: Estimate of renter-occupied units with income > \$150,000 in last year
- HC03_EST_VC14: Estimate of median household income of renter-occupied units in last year

In particular, we'll create two dataframes. One will hold the **percentages** of renter occupied units that fall into each income category, and the other will hold the estimated median household incomes of renter occupied units by tract only. 

### Clean and combine data

In [ ]:
renter_income_only = ["HC03_EST_VC14"]
renter_income_num_homes = ["HC03_EST_VC01"]
renter_income_dist = ["HC03_EST_VC03", "HC03_EST_VC04", "HC03_EST_VC05", "HC03_EST_VC06", 
                      "HC03_EST_VC07", "HC03_EST_VC08", "HC03_EST_VC09", "HC03_EST_VC10", "HC03_EST_VC11",
                      "HC03_EST_VC12", "HC03_EST_VC13"]

df_2016_renter_income_only = df_2016[renter_income_only]
df_2016_renter_income_num_homes = df_2016[renter_income_num_homes]
df_2016_renter_income_dist = df_2016[renter_income_dist]

print(df_2016_renter_income_only.head(5))
print(df_2016_renter_income_num_homes.head(5))

Next, for each entry in ``df_2016_renter_income_dist``, we normalize by the total number of homes in the Census Tract, appearing in the corresponding row of ``df_2016_renter_income_num_homes``. 

In [ ]:
df_2016_renter_income_dist.div(df_2016_renter_income_num_homes, axis = 0).astype(float)
print(df_2016_renter_income_dist.head(5))

Finally, we take the shape of this DataFrame, for future use. 

In [ ]:
print(df_2016_renter_income_dist.shape)

### Develop a "gentrification measure"

We'll let one intuitive measurement for the "wealthiness" of a Census Tract be based on the shape of its income distribution. In particular, we'll use a measurement akin to the Gini Coefficient, where since we lack continuous data, we'll approximate the cumulative share of people from lowest to highest incomes in each tract using the cumulative share of renters with household income up to each income bucket. Let's illustrate as follows. Consider a random Census Tract in the data. 

In [ ]:
tract = int(np.random.rand(1)*df_2016.shape[0])
print(tract)

In [ ]:
incomes = df_2016_renter_income_dist.iloc[tract].astype(float)
print(incomes)

In [ ]:
incomes = np.array(incomes)
cum_incomes = np.cumsum(incomes)
print(cum_incomes)

In [ ]:
tick_labels = ["<5,000", "5,000-9,999", "10,000-14,999", "15,000-19,999", "20,000-24,999", 
               "25,000-34,999", "35,000-49,999", "50,000-74,999", "75,000-99,999", "100,000-149,999", 
               ">150,000"]
tick_labels = [lab + " or less" for lab in tick_labels]
x = np.arange(len(cum_incomes))
plt.bar(x, height = cum_incomes)
plt.plot(x, cum_incomes, color = "red", marker = "o")
plt.title("Cumulative income histogram in Tract {}".format(tract))
plt.xticks(x, labels = tick_labels, rotation = 280)
plt.ylabel("% families renting")
plt.show()

Notice that by taking the cumulative sum of the percentage of renters at each household income bracket, we've transforemd the vector representing the income distribution in the Tract into a monotonically increasing function. To calculate the conventional Gini Coefficient of a society, we would take the ratio between the area of the Lorenz Curve and the area under this monotonically increasing function. 

However, we actually don't have strictly *enough* information to calculate the Gini Coefficient exactly, because we don't know anything about the *cumulative* wealth (income) in the Tract due to the Census's discretization. Also, the last bucket (income > \$150,000) has the potential to be pretty massive, especially in California. So instead, let's consider what the above graph would look like for the Tract if *all* renters earned the minimum amount of household income. Then, the cumulative income histogram would be a box across the domain. 

If we normalize the X-axis coordinates of the bars such that the last bucket's X-axis coordinate is at one, then a "perfectly poor" society would have a Gini Coefficient (equal to the area under the curve divided by the area of the box) of one. Let's let the "transformed" Gini Coefficient for each Census Tract be one minus this "original" value, so that perfectly poor Tracts have Gini scores close to zero and perfectly rich Tracts have Gini scores close to one. 

Some alternative treatments could make this measurement a little more robust, such as e.g. normalizing the X-axis coordinates of each bar using the *upper-bounds* of the income ranges corresponding to those bars. This would spread the red increasing curve out along the X-axis. But, it's a monotonic transformation, and should preserve the *ordering* of Tracts by these Gini scores. 

Let's calculate this new Gini score for the tract under consideration. 

In [ ]:
def calculate_gini_score(incomes):
    incomes = incomes.astype(float)
    cum_incomes = np.cumsum(np.array(incomes))/100
    x = (1 + np.arange(len(cum_incomes)))/len(cum_incomes)
    area = np.trapz(cum_incomes, x = x)
    return (1 - area)

gini = calculate_gini_score(incomes)
print("Gini score of Tract {}: {:.3f}".format(tract, gini))

This relatively high Gini score suggests that this Tract is on the wealthier side -- at least, relative to the fairly arbitrary normalization scheme described above. However, if we inspect the data, we see that the largest increase in the monotonic red function occurs at the point where renters have household incomes around \$50,000, which is not a lot of money in California. 

So, while this scheme will preserve the relative ordering of the Tracts, we may want to increase the variance of the resultant score calculated for each Tract by using the alternate X-axis assignment method described above. In particular, let's see what happens when we calculate this Tract's Gini score by letting the upper-bound of each income bracket be that point's X-coordinate. 

Let's let the X-coordinate of the last bar be \$1M. This is somewhat arbitrary, but it's a nice high number and we really care more about enforcing separation for Tracts with many households whose incomes are concentrated in the middle of the plot. 

In [ ]:
def calculate_gini_score(incomes, x = None):
    try:
        incomes = incomes.astype(float)
    except ValueError:
        return -1
    
    cum_incomes = np.cumsum(np.array(incomes))/100
    if x is None: x = (1 + np.arange(len(cum_incomes)))/len(cum_incomes)
    area = np.trapz(cum_incomes, x = x)/np.max(x)
    return (1 - area)

x = [5000, 9999, 14999, 19999, 24999, 34999, 49999, 74999, 99999, 149999, 1000000]
gini = calculate_gini_score(incomes, x = x)
print("Gini score of Tract {}: {:.3f}".format(tract, gini))

This comports a little better with reality. A Tract where the majority of residents (~80%) earn around \$80,000 in California is... not very wealthy. Finally, let's apply this transformation for every Tract and plot the distribution of the resultant Gini scores to see if we've captured any variance in Census Tract wealth in the state for the year 2016. 

We'll also add one more check to the callback function above. Namely, for some Tracts (where the population is very small), the intercensal estimates are sufficiently poor that the Census reports no data. As we calculate the Gini score for these Tracts, we'll simply assign a placeholder of -1. Then, we'll post-process the data and assign these Tracts the median Gini score in the reported data so they don't affect the resulting distribution. This is somewhat ad-hoc, but is accepted practice where data is missing. 

In [ ]:
df_2016_gini = df_2016_renter_income_dist.apply(lambda a : calculate_gini_score(a), axis = 1)
print(df_2016_gini.head(5))

Next, we tabulate the results to see how many of the Tracts had missing data. We replace the -1 Gini scores we'd assigned to these Tracts previously with the median of the Gini scores for the Tracts where data was present. 

In [ ]:
num_missing = len(df_2016_gini.loc[df_2016_gini == -1])
print(num_missing)

In [ ]:
df_2016_gini[df_2016_gini == -1] = np.median(df_2016_gini[df_2016_gini != -1])
num_missing = len(df_2016_gini.loc[df_2016_gini == -1])
print(num_missing)

In [ ]:
plt.hist(df_2016_gini, bins = 25, rwidth = 0.8)
plt.title("Gini distribution among CA Census Tracts, 2016")
plt.xlabel("Gini score (0-1)")
plt.ylabel("# Tracts")
plt.show()

Hey, that's a nice, pleasant-looking Gaussian distribution. This suggests that we may have constructed a reasonable measurement for a Tract's renter wealth. Finally, let's put this data on a map of California. 

### Plot data

In [ ]:
import geopandas as gpd

shp_path = "tl_2016_06_tract/tl_2016_06_tract.shp"
shp = gpd.read_file(shp_path)
print(shp.head(1))

shp.plot()
plt.xticks([])
plt.yticks([])
plt.title("Census Tracts in California")
plt.show()

Having verified that we have the correct shapefile, we may proceed to join the Gini series by Tract into the new DataFrame, so that we can draw a choropleth. First, we need to check that the index for the Gini series is the column labelled ``GEO.id2`` in the original dataset. Then, we need to assign the same (``GEOID``) column as the index in the shapefile and convert both columns from ``string``s to ``int``s to enable easy joining. Finally, we can join the dataframes by index. 

In [ ]:
new_index = df_2016["GEO.id2"].astype(int)
df_2016_gini = df_2016_gini.reindex(index = new_index)
df_2016_gini.rename("Gini", inplace = True)
print(df_2016_gini.index[:5])

new_index = shp["GEOID"].astype(int)
shp = shp.reindex(index = new_index)
print(shp.index[:5])

In [ ]:
print(df_2016_gini.head(5))
shp = shp.join(df_2016_gini)
print(shp.head(1))

### Predict changes in target variable over time